<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# SAR Single Node on MovieLens (Python, CPU)

In this example, we will walk through each step of the Simple Algorithm for Recommendation (SAR) algorithm using a Python single-node implementation.

SAR is a fast, scalable, adaptive algorithm for personalized recommendations based on user transaction history. It is powered by understanding the similarity between items, and recommending similar items to those a user has an existing affinity for.

## 1 SAR algorithm

The following figure presents a high-level architecture of SAR. 

At a very high level, two intermediate matrices are created and used to generate a set of recommendation scores:

- An item similarity matrix $S$ estimates item-item relationships.
- An affinity matrix $A$ estimates user-item relationships.

Recommendation scores are then created by computing the matrix multiplication $A\times S$.

Optional steps (e.g. "time decay" and "remove seen items") are described in the details below.

<img src="https://recodatasets.blob.core.windows.net/images/sar_schema.svg?sanitize=true">

### 1.1 Compute item co-occurrence and item similarity

SAR defines similarity based on item-to-item co-occurrence data. Co-occurrence is defined as the number of times two items appear together for a given user. We can represent the co-occurrence of all items as a $m\times m$ matrix $C$, where $c_{i,j}$ is the number of times item $i$ occurred with item $j$, and $m$ is the total number of items.

The co-occurence matric $C$ has the following properties:

- It is symmetric, so $c_{i,j} = c_{j,i}$
- It is nonnegative: $c_{i,j} \geq 0$
- The occurrences are at least as large as the co-occurrences. I.e., the largest element for each row (and column) is on the main diagonal: $\forall(i,j) C_{i,i},C_{j,j} \geq C_{i,j}$.

Once we have a co-occurrence matrix, an item similarity matrix $S$ can be obtained by rescaling the co-occurrences according to a given metric. Options for the metric include `Jaccard`, `lift`, and `counts` (meaning no rescaling).


If $c_{ii}$ and $c_{jj}$ are the $i$th and $j$th diagonal elements of $C$, the rescaling options are:

- `Jaccard`: $s_{ij}=\frac{c_{ij}}{(c_{ii}+c_{jj}-c_{ij})}$
- `lift`: $s_{ij}=\frac{c_{ij}}{(c_{ii} \times c_{jj})}$
- `counts`: $s_{ij}=c_{ij}$

In general, using `counts` as a similarity metric favours predictability, meaning that the most popular items will be recommended most of the time. `lift` by contrast favours discoverability/serendipity: an item that is less popular overall but highly favoured by a small subset of users is more likely to be recommended. `Jaccard` is a compromise between the two.


### 1.2 Compute user affinity scores

The affinity matrix in SAR captures the strength of the relationship between each individual user and the items that user has already interacted with. SAR incorporates two factors that can impact users' affinities: 

- It can consider information about the **type** of user-item interaction through differential weighting of different events (e.g. it may weigh events in which a user rated a particular item more heavily than events in which a user viewed the item).
- It can consider information about **when** a user-item event occurred (e.g. it may discount the value of events that take place in the distant past.

Formalizing these factors produces us an expression for user-item affinity:

$$a_{ij}=\sum_k w_k \left(\frac{1}{2}\right)^{\frac{t_0-t_k}{T}} $$

where the affinity $a_{ij}$ for user $i$ and item $j$ is the weighted sum of all $k$ events involving user $i$ and item $j$. $w_k$ represents the weight of a particular event, and the power of 2 term reflects the temporally-discounted event. The $(\frac{1}{2})^n$ scaling factor causes the parameter $T$ to serve as a half-life: events $T$ units before $t_0$ will be given half the weight as those taking place at $t_0$.

Repeating this computation for all $n$ users and $m$ items results in an $n\times m$ matrix $A$. Simplifications of the above expression can be obtained by setting all the weights equal to 1 (effectively ignoring event types), or by setting the half-life parameter $T$ to infinity (ignoring transaction times).

### 1.3 Remove seen item

Optionally we remove items which have already been seen in the training set, i.e. don't recommend items which have been previously bought by the user again.

### 1.4 Top-k item calculation

The personalized recommendations for a set of users can then be obtained by multiplying the affinity matrix ($A$) by the similarity matrix ($S$). The result is a recommendation score matrix, where each row corresponds to a user, each column corresponds to an item, and each entry corresponds to a user / item pair. Higher scores correspond to more strongly recommended items.

It is worth noting that the complexity of recommending operation depends on the data size. SAR algorithm itself has $O(n^3)$ complexity. Therefore the single-node implementation is not supposed to handle large dataset in a scalable manner. Whenever one uses the algorithm, it is recommended to run with sufficiently large memory. 

## 2 SAR single-node implementation

The SAR implementation illustrated in this notebook was developed in Python, primarily with Python packages like `numpy`, `pandas`, and `scipy` which are commonly used in most of the data analytics / machine learning tasks. Details of the implementation can be found in [Recommenders/reco_utils/recommender/sar/sar_singlenode.py](../../reco_utils/recommender/sar/sar_singlenode.py).

## 3 SAR single-node based movie recommender

In [3]:
# set the environment path to find Recommenders
import sys
sys.path.append("../")

import itertools

import logging
import os

import numpy as np
import pandas as pd
import papermill as pm

from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_stratified_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.7.3 (default, Oct  7 2019, 12:56:13) 
[GCC 8.3.0]
Pandas version: 0.25.1


In [4]:
# top k items to recommend
TOP_K = 10

### 3.1 Load Data

SAR is intended to be used on interactions with the following schema:
`<User ID>, <Item ID>, <Time>`. 

Each row represents a single interaction between a user and an item. These interactions might be different types of events on an e-commerce website, such as a user clicking to view an item, adding it to a shopping basket, following a recommendation link, and so on. 

The MovieLens dataset is well formatted interactions of Users providing Ratings to Movies (movie ratings are used as the event weight) - we will use it for the rest of the example.

In [5]:
data_dir = '../data'
data_type = '/synthetic' #original or synthetic
data_path = data_dir + data_type
review_ratio = .5
n_users = 20
n_recipes_per_cuisine = 50

#review_fn = '/recipe_reviews.csv'
#features_fn = '/recipe_features.csv'

review_fn = '/reviews/{}_users_dynamicratio.csv'.format(n_users)
features_fn = '/recipes/cuisine_size_{}.csv'.format(n_recipes_per_cuisine)

In [6]:

data = pd.read_csv(data_path + review_fn)# encoding = "ISO-8859-1",delimiter="|")
features = pd.read_csv(data_path + features_fn)#,encoding = "ISO-8859-1",delimiter="|")

# only keep recipes with reviews
features = pd.merge(features, data, left_on='recipe_id', right_on='recipe_id', how="inner")
features = features[["recipe_id", "cuisine", "clean_ingredients"]]
features = features.drop_duplicates().reset_index()
features = features[["recipe_id", "cuisine", "clean_ingredients"]]

# Convert ingredients column to list
features["clean_ingredients"] = features["clean_ingredients"].apply(lambda a : a.split("+"))

# Convert the float precision to 32-bit in order to reduce memory consumption 
data.loc[:, 'rating'] = data['rating'].astype(np.float32)


In [7]:
features.head()

,recipe_id,cuisine,clean_ingredients
0,Pork-Schnitzels-778938,German,"[crumbs, chops, pork, bread, panko, flour, egg..."
1,Sausage-Sauerkraut-Sloppy-Joes-1557576,German,"[onion, rolls, had, ground, sauerkraut, cheese..."
2,Sauerkraut-Chickpea-Flour-Ravioli-_-Spiced-App...,German,"[onion, husk, powder, unsweetened, psyllium, s..."
3,Sauerbraten-1500604,German,"[onion, black, lowsodium, cookies, allspice, t..."
4,Bratwurst-With-Apples_-Onion_-And-Sauerkraut-E...,German,"[black, onion, lengthwise, golden, apples, bra..."


In [8]:
data.head()

,Unnamed: 0,username,recipe_id,rating
0,21,lurline_needs_german_eatz_697,German_style-Pasta-Recipezaar,5.0
1,9,lurline_needs_german_eatz_697,Bratwurst-With-Apples_-Onion_-And-Sauerkraut-E...,5.0
2,36,lurline_needs_german_eatz_697,Bierrocks-Ii-Allrecipes,5.0
3,40,lurline_needs_german_eatz_697,Pork-Chops-German-Style-Allrecipes,5.0
4,33,lurline_needs_german_eatz_697,Sauerbraten-Recipezaar_6,5.0


In [9]:
#print(data.columns)
features.describe()

,recipe_id,cuisine,clean_ingredients
count,711,711,711
unique,711,21,696
top,Merlot-Pot-Roast-With-Horseradish-Smashed-Pota...,Spanish,"[oil, chicken, vegetable, selfrising, buttermi..."
freq,1,47,3


### 3.2 Split the data using the python random splitter provided in utilities:

We split the full dataset into a `train` and `test` dataset to evaluate performance of the algorithm against a held-out set not seen during training. Because SAR generates recommendations based on user preferences, all users that are in the test set must also exist in the training set. For this case, we can use the provided `python_stratified_split` function which holds out a percentage (in this case 25%) of items from each user, but ensures all users are in both `train` and `test` datasets. Other options are available in the `dataset.python_splitters` module which provide more control over how the split occurs.


In [10]:
header = {
    "col_user": "username",
    "col_item": "recipe_id",
    "col_rating": "rating",
    "col_timestamp": "date",
    "col_prediction": "Prediction",
}

In [11]:
train, test = python_stratified_split(data, ratio=0.80, col_user=header["col_user"], col_item=header["col_item"],seed=42)
for r in features["recipe_id"]:
    dummy = pd.DataFrame([["dummy",r,3]], columns=['username',"recipe_id","rating"])
    train = train.append(dummy, ignore_index=True)
print(train.shape)
print(test.shape)

/home/anton/.local/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


(1671, 4)
(240, 4)


In this case, for the illustration purpose, the following parameter values are used:

|Parameter|Value|Description|
|---------|---------|-------------|
|`similarity_type`|`jaccard`|Method used to calculate item similarity.|
|`time_decay_coefficient`|30|Period in days (term of $T$ shown in the formula of Section 2.2.2)|
|`time_now`|`None`|Time decay reference.|
|`timedecay_formula`|`True`|Whether time decay formula is used.|

In [12]:
#V1 (key error)
# set log level to INFO

#from reco_utils.recommender.sar.sar_singlenode import SARSingleNode

#logging.basicConfig(level=logging.DEBUG, 
#                    format='%(asctime)s %(levelname)-8s %(message)s')

#model = SARSingleNode(
#    recipes=features, #FIX
#    similarity_type="custom", 
#    time_decay_coefficient=30, 
#    time_now=None, 
#    timedecay_formula=False, #add recipes to node CAN SET THIS TO TRUE LATER BUT LETS LEAVE IT FOR NOW
#    **header
#)


#model.fit(train)

In [13]:
features

,recipe_id,cuisine,clean_ingredients
0,Pork-Schnitzels-778938,German,"[crumbs, chops, pork, bread, panko, flour, egg..."
1,Sausage-Sauerkraut-Sloppy-Joes-1557576,German,"[onion, rolls, had, ground, sauerkraut, cheese..."
2,Sauerkraut-Chickpea-Flour-Ravioli-_-Spiced-App...,German,"[onion, husk, powder, unsweetened, psyllium, s..."
3,Sauerbraten-1500604,German,"[onion, black, lowsodium, cookies, allspice, t..."
4,Bratwurst-With-Apples_-Onion_-And-Sauerkraut-E...,German,"[black, onion, lengthwise, golden, apples, bra..."
...,...,...,...
706,Make-Low-Sodium-Spaghetti-Sauce-1702290,Italian,"[onion, black, good, wine, olive, ground, flat..."
707,Fennel-Pork-Meatball-578332,Italian,"[breadcrumbs, onion, leaf, olive, ground, egg,..."
708,Sausage_-Spinach-and-Ricotta-Stuffed-Pasta-She...,Italian,"[onion, container, mushrooms, seasoning, olive..."
709,Gnocchi-alla-Sorrentina-1786441,Italian,"[potatoes, oil, tomatoes, pulp, egg, saltpeppe..."


In [23]:
#V2
# set log level to INFO
from reco_utils_2.recommender.sar.sar_singlenode import SARSingleNode

logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

model = SARSingleNode(
    similarity_type="custom", 
    time_decay_coefficient=30, 
    time_now=None, 
    timedecay_formula=False, #add recipes to node CAN SET THIS TO TRUE LATER BUT LETS LEAVE IT FOR NOW
    **header
)

jaccard = lambda a,b: len(set(a).intersection(set(b)))/len(set(a).union(set(b)))

model.fit(train, features, "recipe_id", {"ratings" : 1})#, "clean_ingredients" : (0.5, jaccard)})

2019-12-09 21:57:53,866 INFO     Collecting user affinity matrix
2019-12-09 21:57:53,869 INFO     De-duplicating the user-item counts
2019-12-09 21:57:53,873 INFO     Creating index columns
2019-12-09 21:57:53,889 INFO     Building user affinity sparse matrix
2019-12-09 21:57:53,893 INFO     Calculating item co-occurrence
2019-12-09 21:57:53,928 INFO     Calculating item similarity
2019-12-09 21:57:53,944 INFO     Done training


items in df 1671
index to item len 711
item to index len 711


In [19]:

#model.save_to_file()

In [20]:
#execute this to load model

#from reco_utils_2.recommender.sar.sar_singlenode import SARSingleNode

#logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

#model = SARSingleNode(
#    similarity_type="custom", 
#    time_decay_coefficient=30, 
    time_now=None, 
    timedecay_formula=False, #add recipes to node CAN SET THIS TO TRUE LATER BUT LETS LEAVE IT FOR NOW
    **header
)

model.fit(train, features, "recipe_id", None, load=True)

IndentationError: unexpected indent (<ipython-input-20-1457119ff497>, line 6)

In [24]:
from reco_utils_2.evaluation.custom_evaluation import accuracy_metric

for i in [1, 3, 5, 10]:
    absolute, relative = accuracy_metric(model, test, i)
    print(i, "absolute:", absolute, "\trelative:", relative)

2019-12-09 21:57:58,095 INFO     Calculating recommendation scores
2019-12-09 21:57:58,106 INFO     Removing seen items
2019-12-09 21:57:58,111 WARNING  Number of items is less than top_k, limiting top_k to number of items
2019-12-09 21:57:58,267 INFO     Calculating recommendation scores
2019-12-09 21:57:58,269 INFO     Removing seen items
2019-12-09 21:57:58,271 WARNING  Number of items is less than top_k, limiting top_k to number of items
2019-12-09 21:57:58,370 INFO     Calculating recommendation scores
2019-12-09 21:57:58,372 INFO     Removing seen items
2019-12-09 21:57:58,373 WARNING  Number of items is less than top_k, limiting top_k to number of items


1 absolute: 0.4 	relative: 4.800000000000001
3 absolute: 0.45 	relative: 1.8


2019-12-09 21:57:58,485 INFO     Calculating recommendation scores
2019-12-09 21:57:58,487 INFO     Removing seen items
2019-12-09 21:57:58,489 WARNING  Number of items is less than top_k, limiting top_k to number of items


5 absolute: 0.52 	relative: 1.248
10 absolute: 0.865 	relative: 1.038


In [165]:
top_k = model.recommend_k_items(train, remove_seen=True)
top_k.head()

2019-12-08 21:06:42,177 INFO     Calculating recommendation scores
2019-12-08 21:06:42,179 INFO     Removing seen items


,username,recipe_id,Prediction
0,chinese_finesser_593,Dhal-Curry-983208,60.871190
1,chinese_finesser_593,Greek-Stuffed-Tomatoes-With-Quinoa-And-Black-L...,60.337195
2,chinese_finesser_593,Steamed-Scallops-with-Soy-Sauce-and-Garlic-Oil...,58.298650
3,chinese_finesser_593,Spicy-Moroccan-Shrimp-Tagine-513928,57.690683
4,chinese_finesser_593,Thai-Red-Curry-Meatballs-with-Coconut-Curry-Sa...,57.348882


The final output from the `recommend_k_items` method generates recommendation scores for each user-item pair, which are shown as follows.

In [166]:
top_k_with_titles = (top_k.join(data[['recipe_id']].drop_duplicates().set_index('recipe_id'), 
                                on='recipe_id', 
                                how='inner').sort_values(by=['username', 'Prediction'], ascending=False))
display(top_k_with_titles.head(10))

,username,recipe_id,Prediction
90,trang_only_eats_chinese_food_076,English-Roast-Beef-Allrecipes,65.789723
91,trang_only_eats_chinese_food_076,Spanish-Meatballs-_Albondigas_-with-Pintos_-Zu...,64.568017
92,trang_only_eats_chinese_food_076,Moroccan-Spiced-Fish-1994862,64.183143
93,trang_only_eats_chinese_food_076,Stuffed-Grape-Leaves-with-Cucumber-Yogurt-Sauc...,63.909300
94,trang_only_eats_chinese_food_076,Tofu-Scramble-Recipe-Southwestern-Style-1994866,63.123103
95,trang_only_eats_chinese_food_076,Vegan-Irish-Hand-Pies-1894258,62.451949
96,trang_only_eats_chinese_food_076,Thai-Red-Curry-Meatballs-with-Coconut-Curry-Sa...,61.285298
97,trang_only_eats_chinese_food_076,Spicy-Moroccan-Shrimp-Tagine-513928,61.187768
98,trang_only_eats_chinese_food_076,Sauerbraten-Beef-in-Gingersnap-Gravy-1540117,61.146492
99,trang_only_eats_chinese_food_076,Spanish-rice-with-chicken-299070,61.071635


### 3.3 Evaluate the results

It should be known that the recommendation scores generated by multiplying the item similarity matrix $S$ and the user affinity matrix $A$ **DOES NOT** have the same scale with the original explicit ratings in the movielens dataset. That is to say, SAR algorithm is meant for the task of *recommending relevent items to users* rather than *predicting explicit ratings for user-item pairs*. 

To this end, ranking metrics like precision@k, recall@k, etc., are more applicable to evaluate SAR algorithm. The following illustrates how to evaluate SAR model by using the evaluation functions provided in the `reco_utils`.

In [167]:
# all ranking metrics have the same arguments
args = [test, top_k]
kwargs = dict(col_user='username', 
              col_item='recipe_id', 
              col_rating='rating', 
              col_prediction='Prediction', 
              relevancy_method='top_k', 
              k=TOP_K)

eval_map = map_at_k(*args, **kwargs)
eval_ndcg = ndcg_at_k(*args, **kwargs)
eval_precision = precision_at_k(*args, **kwargs)
eval_recall = recall_at_k(*args, **kwargs)

In [168]:
print("Model:", 
      "Top K:\t\t {}".format(top_k),
      "MAP:\t\t {}".format(eval_map),
      "NDCG:\t\t {}".format(eval_ndcg),
      "Precision@K:\t {}".format(eval_precision),
      "Recall@K:\t {}".format(eval_recall),sep='\n')

# print("Model:",
#       f"Top K:\t\t {TOP_K:f}",
#       f"MAP:\t\t {eval_map:f}",
#       f"NDCG:\t\t {eval_ndcg:f}",
#       f"Precision@K:\t {eval_precision:f}",
#       f"Recall@K:\t {eval_recall:f}", sep='\n')

Model:
Top K:		                             username  \
0               chinese_finesser_593   
1               chinese_finesser_593   
2               chinese_finesser_593   
3               chinese_finesser_593   
4               chinese_finesser_593   
..                               ...   
95  trang_only_eats_chinese_food_076   
96  trang_only_eats_chinese_food_076   
97  trang_only_eats_chinese_food_076   
98  trang_only_eats_chinese_food_076   
99  trang_only_eats_chinese_food_076   

                                            recipe_id  Prediction  
0                                   Dhal-Curry-983208   60.871190  
1   Greek-Stuffed-Tomatoes-With-Quinoa-And-Black-L...   60.337195  
2   Steamed-Scallops-with-Soy-Sauce-and-Garlic-Oil...   58.298650  
3                 Spicy-Moroccan-Shrimp-Tagine-513928   57.690683  
4   Thai-Red-Curry-Meatballs-with-Coconut-Curry-Sa...   57.348882  
..                                                ...         ...  
95                      Veg

## References
Note SAR is a combinational algorithm that implements different industry heuristics. The followings are references that may be helpful in understanding the SAR logic and implementation. 

1. Badrul Sarwar, *et al*, "Item-based collaborative filtering recommendation algorithms", WWW, 2001.
2. Scipy (sparse matrix), url: https://docs.scipy.org/doc/scipy/reference/sparse.html
3. Asela Gunawardana and Guy Shani, "A survey of accuracy evaluation metrics of recommendation tasks", The Journal of Machine Learning Research, vol. 10, pp 2935-2962, 2009.	